# Repair Poorly Design Contract So It Runs on the Blockchain


**Executive Summary**

We use the `marlowe-cli run analyze` command to demonstrate how to detect problems in a Marlowe contract that would prevent it from successfully running on the blockchain. This tool detects all known contract-design issues that would prevent a contract from executing successfully. It checks the Marlowe initial state, the validity of role and non-role token names, the minimum-UTxO ledger rule, the maximum-Value ledger rule, execution cost, and transaction size. These checks do the replace static analysis, which focuses on *semantics* instead of realizability on the blockchain.

The table below summarizes the situation for Marlowe contracts that might violate the Cardano protocol parameters. Items are marked "locks" if they would prevent further execution of the contract if they are the top-level construct or the timeout continuation of a `When` contract; otherwise, they just block some (but not all) execution paths for the contract. They are marked "blocks" if they can never lock the contract.

| Protocol Limit   | Protocol Parameter           | `Pay` | `Let` | `If`/`Assert` | `When` | `Close` | `Deposit` | `Choice` | `Notify` |
|------------------|------------------------------|-------|-------|---------------|--------|---------|-----------|----------|----------|
| Transaction size | `maxTxSize`                  | locks | locks |               | blocks | locks   |           | blocks   |          |
| Execution steps  | `maxTxExecutionUnits.steps`  | locks | locks | locks         | blocks | locks   | blocks    | blocks   | blocks   |
| Execution memory | `maxTxExecutionUnits.memory` | locks | locks | locks         | blocks | locks   | blocks    | blocks   | blocks   |
| Minimum ADA      | `utxoCostPerWord`            |       |       |               |        |         | blocks    |          |          |
| Value size       | `maxValueSize`               | locks |       |               |        |         | blocks    |          |          |

## The "Analyze" Command in Marlowe CLI

In [1]:
marlowe-cli run analyze --help

Usage: marlowe-cli run analyze --testnet-magic INTEGER --socket-path SOCKET_FILE
                               --marlowe-file MARLOWE_FILE [--preconditions] 
                               [--roles] [--tokens] [--maximum-value] 
                               [--minimum-utxo] [--execution-cost] 
                               [--transaction-size] [--best] [--verbose]

  [EXPERIMENTAL] Analyze a Marlowe contract.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                           Location of the cardano-node socket file. Defaults to
                           the CARDANO_NODE_SOCKET_PATH environment variable's
                           value.
  --marlowe-file MARLOWE_FILE
                           JSON file with the state and contract.
  --preconditions          Whether to check preconditions for valid Marlowe
                           state

## Preliminaries

### Record versions

In [2]:
marlowe-cli --version

marlowe-cli 0.0.10.0


In [3]:
z3 --version

Z3 version 4.8.12 - 64 bit


In [4]:
git rev-parse HEAD

9380e9d24d2d0876c250d0008b4e48caf9c8238e


### Select the network

In [5]:
export CARDANO_TESTNET_MAGIC=1

In [6]:
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/preprod/node.socket

## Create the contract

In [7]:
yaml2json << EOI > repair-0.contract
when:
- case:
    party:
      role_token: This party has a role name that is greater than the 32-byte limit.  # This is too long.
    deposits: 100000000
    of_token:
      currency_symbol: ""  # Ada has a blank currency symbol.
      token_name: ""       # Ada has a blank token name.
    into_account:
      role_token: ""       # It's legal to have a blank role name, but this may have unintended consequences during payout.
  then:
    pay: 10000000
    token:
      currency_symbol: ""     # A blank currency symbol may not have
      token_name: "Blank CS"  # a non-blank token name.
    to:
      party:
        role_token: "Party 1"
    from_account:
      role_token: "Party 0"
    then:
      pay: 10000000
      token:
        currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c533  # This is too short by one byte.
        token_name: CS too short
      to:
        party:
          role_token: Party 2
      from_account:
        role_token: Party 0
      then:
        pay: 10000000
        token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d00  # This is too long by one byte.
          token_name: CS too long
        to:
          party:
            role_token: Party 3
        from_account:
          role_token: Party 0
        then:
          pay: 10000000
          token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: This name is far too long for a token name, which should be 32 bytes or less.  # This is too long.
          to:
            party:
              role_token: Party 4
          from_account:
            role_token: Party 0
          then:
            pay: 10000000
            token:
              currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
              token_name: ""  # It okay to have a blank token name.
            to:
              party:
                role_token: Party 5
            from_account:
              role_token: Party 0
            then: close
timeout_continuation : close
timeout: 1665266660244
EOI

cat repair-0.contract

{"timeout":1665266660244,"timeout_continuation":"close","when":[{"case":{"deposits":100000000,"into_account":{"role_token":""},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"This party has a role name that is greater than the 32-byte limit."}},"then":{"from_account":{"role_token":"Party 0"},"pay":10000000,"then":{"from_account":{"role_token":"Party 0"},"pay":10000000,"then":{"from_account":{"role_token":"Party 0"},"pay":10000000,"then":{"from_account":{"role_token":"Party 0"},"pay":10000000,"then":{"from_account":{"role_token":"Party 0"},"pay":10000000,"then":"close","to":{"party":{"role_token":"Party 5"}},"token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":""}},"to":{"party":{"role_token":"Party 4"}},"token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"This name is far too long for a token name, which should be 32 bytes or less."}},"to":{"party":{"role_token":"Party 

## Create the initial state

In [8]:
yaml2json << EOI > repair-0.state
accounts:
- - - role_token: Someone
    - currency_symbol: 0ada00000000000000000000000000000000000000000000000000000000000000  # This is too long.
      token_name: ""
  - 0  # This is zero
- - - role_token: This role token name is a little too long.  # This is too long.
    - currency_symbol: ""
      token_name: A token
  - 300
- - - role_token: Someone
    - currency_symbol: 1ada
      token_name: A far, far, too very long name for a native token.
  - 300
- - - role_token: Someone                                             # This is a duplicate entry.
    - currency_symbol: 1ada                                           # This is too short.
      token_name: A far, far, too very long name for a native token.  # This is too long.
  - 330
- - - address: addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q
    - currency_symbol: ""
      token_name: ""
  - -3  # This is negative.
boundValues:
- - some value
  - 3
- - some value  # This is a duplicate entry.
  - 33
choices:
- - choice_name: some choice
    choice_owner:
      role_token: Party
  - 10
- - choice_name: some choice  # This is a duplicate choice.
    choice_owner:
      role_token: Party
  - 101
- - choice_name: some choice
    choice_owner:
      role_token: Another role name that is too long.  # This is too long.
  - 101
minTime: 1
EOI

cat repair-0.state

{"accounts":[[[{"role_token":"Someone"},{"currency_symbol":"0ada00000000000000000000000000000000000000000000000000000000000000","token_name":""}],0],[[{"role_token":"This role token name is a little too long."},{"currency_symbol":"","token_name":"A token"}],300],[[{"role_token":"Someone"},{"currency_symbol":"1ada","token_name":"A far, far, too very long name for a native token."}],300],[[{"role_token":"Someone"},{"currency_symbol":"1ada","token_name":"A far, far, too very long name for a native token."}],330],[[{"address":"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q"},{"currency_symbol":"","token_name":""}],-3]],"boundValues":[["some value",3],["some value",33]],"choices":[[{"choice_name":"some choice","choice_owner":{"role_token":"Party"}},10],[{"choice_name":"some choice","choice_owner":{"role_token":"Party"}},101],[{"choice_name":"some choice","choice_owner":{"role_token":"Another role name that is too long."}},101]],"minTime":1}


## Bundle the contract and state into a single file

In [9]:
marlowe-cli run initialize \
  --roles-currency "ada1" \
  --contract-file repair-0.contract \
  --state-file    repair-0.state \
  --out-file      repair-0.marlowe

## Check the role names

The tool detects the following problems:
1. One of the role names is blank.
2. One role names is longer than 32 bytes, which means it won't fit on the blockchain.

In [10]:
marlowe-cli run analyze \
  --marlowe-file repair-0.marlowe \
  --roles

- Role names:
    Blank role names: true
    Invalid role names:
    - unTokenName: This party has a role name that is greater than the 32-byte limit.


## Check the token currency symbols and names

The tool detects the following problems:
1. A token has a blank (Ada-like) currency symbol, but a non-blank (not Ada) token name.
2. One of the tokens has a currency symbol that is shorter than 32 bytes.
3. One of the tokens has a currency symbol that is longer than 32 bytes.

In [11]:
marlowe-cli run analyze \
  --marlowe-file repair-0.marlowe \
  --tokens

- Tokens:
    Invalid tokens:
    - currency_symbol: ''
      token_name: Blank CS
    - currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c533
      token_name: CS too short
    - currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: This name is far too long for a token name, which should be 32 bytes
        or less.
    - currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d00
      token_name: CS too long


## Check the initial state

The tool detects the following problems:
1. There is a duplicate account entry.
2. There is a duplicate bound value.
3. There is a duplice choice.
4. The role name in one of the accounts exceeds the 32-byte limit.
5. The accounts have currency simbols that are too short or too long.
6. A choice has an owner whose role name is too long.
7. The roles currency is not a valid currency symbole.
8. There are non-positive account balances.

In [12]:
marlowe-cli run analyze \
  --marlowe-file repair-0.marlowe \
  --preconditions

- Preconditions:
    Duplicate accounts:
    - - role_token: Someone
      - currency_symbol: 1ada
        token_name: A far, far, too very long name for a native token.
    Duplicate bound values:
    - some value
    Duplicate choices:
    - choice_name: some choice
      choice_owner:
        role_token: Party
    Invalid account parties:
    - role_token: This role token name is a little too long.
    Invalid account tokens:
    - currency_symbol: 0ada00000000000000000000000000000000000000000000000000000000000000
      token_name: ''
    - currency_symbol: ''
      token_name: A token
    - currency_symbol: 1ada
      token_name: A far, far, too very long name for a native token.
    - currency_symbol: 1ada
      token_name: A far, far, too very long name for a native token.
    Invalid choice parties:
    - choice_name: some choice
      choice_owner:
        role_token: Another role name that is too long.
    Invalid roles currency: true
    Non-positive account balances:
    - -

## Fix the contract and its initial state.

In [13]:
yaml2json << EOI > repair-1.contract
when:
- case:
    party:
      address: addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chhlhkzy8tlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsq859x2g
    deposits:
      multiply:
        multiply: 2
        times:
          amount_of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A longer token name.
          in_account:
            address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
      times:
        amount_of_token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name.
        in_account:
          address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name.
    into_account:
      address: addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps
  then: close
- case:
    party:
      address: addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chhlhkzy8tlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsq859x2g
    deposits:
      multiply:
        multiply: 3
        times:
          amount_of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A longer token name.
          in_account:
            address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
      times:
        amount_of_token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name.
        in_account:
          address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name.
    into_account:
      address: addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps
  then: close
- case:
    party:
      address: addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chhlhkzy8tlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsq859x2g
    deposits:
      multiply:
        multiply: 4
        times:
          amount_of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A longer token name.
          in_account:
            address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
      times:
        amount_of_token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name.
        in_account:
          address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name.
    into_account:
      address: addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps
  then: close
- case:
    party:
      address: addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chhlhkzy8tlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsq859x2g
    deposits:
      multiply:
        multiply: 5
        times:
          amount_of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A longer token name.
          in_account:
            address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
      times:
        amount_of_token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name.
        in_account:
          address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name.
    into_account:
      address: addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps
  then: close
- case:
    party:
      address: addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chhlhkzy8tlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsq859x2g
    deposits:
      multiply:
        multiply: 6
        times:
          amount_of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A longer token name.
          in_account:
            address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
      times:
        amount_of_token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name.
        in_account:
          address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name.
    into_account:
      address: addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps
  then: close
- case:
    party:
      address: addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chhlhkzy8tlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsq859x2g
    deposits:
      multiply:
        multiply: 7
        times:
          amount_of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A longer token name.
          in_account:
            address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
      times:
        amount_of_token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name.
        in_account:
          address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name.
    into_account:
      address: addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps
  then: close
- case:
    party:
      role_token: This party has a long role name.
    deposits: 100000000
    of_token:
      currency_symbol: ""
      token_name: ""
    into_account:
      role_token: Someone
  then:
    pay: 10000000
    token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long token name does use bytes
    to:
      party:
        address: addr_test1qqhsrfkwq3pzwftdd6r8hhent88p5c8t9y6vp74450wm7xkdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vq2pxm5e
    from_account:
      address: addr_test1qp7y79n8052fvt7a6wj4ehst532mgz4rep09hxej980vmv7dtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqenynfn
    then:
      pay: 10000000
      token:
        currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
        token_name: A long token name does use bytes
      to:
        party:
          address: addr_test1qq5am6hqgnw9u477a4eurm9zyr7rtkje27y4l93f9qfyulkdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vq34clrj
      from_account:
        address: addr_test1qqu6pjzr6k3lvfx3xarey7jwlwtg2mlz3gypwk709y48g2wdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vquh9ja4
      then:
        pay: 10000000
        token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: A long token name does use bytes
        to:
          party:
            address: addr_test1qq2neemmna2urqjx8tupuqh9qnur8gfaftqhtvtt5qxscgkdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vq8pjzcd
        from_account:
          address: addr_test1qqu8ycccfm0lsxp6m6zhpqp3hulrv94axvpexsk5ds0e037dtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vq4lqvjw
        then:
          pay: 10000000
          token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
            token_name: A long token name does use bytes
          to:
            party:
              address: addr_test1qqv2qyg89kmnxuf7yssl8ysfdtmhqqs4cx2negy6f4xx68xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfys92
          from_account:
            address: addr_test1qpgdhv928sznqhafs0pskus4wzjj9m6le45a0f3vtlzddvxdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqr70phm
          then:
            pay: 10000000
            token:
              currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
              token_name: A long token name does use bytes
            to:
              party:
                address: addr_test1qqtd3kz2x7tdwaqdcn46y2k8vjlcqeh3jswxhc54arujfe7dtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqcsq6nj
            from_account:
              address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
            then: 
              assert:
                either:
                  chose_something_for:
                    choice_name: name
                    choice_owner:
                      address: addr_test1qrplmzeprfkhszygte7qff9w4pxdr6qkxmak80ccqf3jyywdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqwlrrnv
                or:
                  le_than:
                    multiply: 123456789
                    times: 987654321
                  value:
                    amount_of_token:
                      currency_symbol: ''
                      token_name: ''
                    in_account:
                      address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
              then: close
timeout_continuation : close
timeout: 1665266660244
EOI

cat repair-1.contract

{"timeout":1665266660244,"timeout_continuation":"close","when":[{"case":{"deposits":{"multiply":{"multiply":2,"times":{"amount_of_token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"A longer token name."},"in_account":{"address":"addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel"}}},"times":{"amount_of_token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"A long token name."},"in_account":{"address":"addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel"}}},"into_account":{"address":"addr_test1qqfaz90xvyc24p2xm2yp08402v3hdu38685nxklrssl5ypnlxyxn4wsn2z3watr72naayj7kctvygade83cw98kd8gsqm9lrps"},"of_token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"A long token name."},"party":{"address":"addr_test1qqqfx893k998ncu0ukl9ccll84adazyngemf6chh

In [14]:
yaml2json << EOI > repair-1.state
accounts:
- - - role_token: Someone
    - currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: ""
  - 1
- - - role_token: This role token name long.
    - currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A token
  - 300
- - - role_token: Someone
    - currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: A long name for a native token.
  - 300
- - - address: addr_test1qrplmzeprfkhszygte7qff9w4pxdr6qkxmak80ccqf3jyywdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqwlrrnv
    - currency_symbol: ""
      token_name: ""
  - 3
boundValues:
- - some value
  - 3
choices:
- - choice_name: some choice
    choice_owner:
      role_token: Party
  - 10
- - choice_name: some choice
    choice_owner:
      role_token: Another long role name.
  - 101
minTime: 1
EOI

cat repair-1.state

{"accounts":[[[{"role_token":"Someone"},{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":""}],1],[[{"role_token":"This role token name long."},{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"A token"}],300],[[{"role_token":"Someone"},{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"A long name for a native token."}],300],[[{"address":"addr_test1qrplmzeprfkhszygte7qff9w4pxdr6qkxmak80ccqf3jyywdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqwlrrnv"},{"currency_symbol":"","token_name":""}],3]],"boundValues":[["some value",3]],"choices":[[{"choice_name":"some choice","choice_owner":{"role_token":"Party"}},10],[{"choice_name":"some choice","choice_owner":{"role_token":"Another long role name."}},101]],"minTime":1}


## Check that the problems have been fixed

Bundle the contract and initial state.

In [15]:
marlowe-cli run initialize \
  --roles-currency 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d \
  --contract-file repair-1.contract \
  --state-file    repair-1.state \
  --out-file      repair-1.marlowe

Re-run the analysis.

In [16]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --roles \
  --tokens \
  --preconditions

- Preconditions:
    Duplicate accounts: []
    Duplicate bound values: []
    Duplicate choices: []
    Invalid account parties: []
    Invalid account tokens: []
    Invalid choice parties: []
    Invalid roles currency: false
    Non-positive account balances: []
- Role names:
    Blank role names: false
    Invalid role names: []
- Tokens:
    Invalid tokens: []


## Check whether the contract satisfies the ledger's "minimum UTxO" requirement

The `minUTxO` requirements specifies that the minimum Ada that must be present at a UTxO, given the number of bytes stored there. The initial state of the contract must contain at least this much Ada.

Without the `--best` flag, the analysis tool computes a rough upper bound on the minimum UTxO Ada value.

In [17]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --minimum-utxo

- Minimum UTxO:
    Requirement:
      lovelace: 1594700


With the `--best` flag, the analysis tool visits every execution path in the contract and finds the maximum of the minimum UTxO value over all paths.

In [18]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --minimum-utxo \
  --best

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 3 execution paths.
- Minimum UTxO:
    Requirement:
      lovelace: 1594700


The `--verbose` flag will print where the worst case occurs in the contract.

In [19]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --minimum-utxo \
  --best \
  --verbose

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 3 execution paths.
- Minimum UTxO:
    Requirement:
      lovelace: 1594700
    Worst case:
      contract:
        timeout: 1665266660244
        timeout_continuation: close
        when:
        - case:
            deposits:
              multiply:
                multiply: 2
                times:
                  amount_of_token:
                    currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
                    token_name: A longer token name.
                  in_account:
                    address: addr_test1qp7cz0ssxs7cmllcdsxlgcdsg940n2xmcfuuy98g5lk7v0xdtcmjhx29ptfrcx5m4y57rmjg3vhwcrte02f5mn55l5vqdfztel
              times:
                amount_of_token:
                  currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
                  token_name: A long 

## Check whether the contract satisfies the ledger's "maximum value" requirement

The `maxValue` requirement specifies that a maximum of 625 words of data describing native tokens can be stored in a UTxO.

Without the `--best` flag, the analysis tool computes a rough upper bound on the maximum number of words needed to store native tokens in the contract.

In [20]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --maximum-value

- Maximum value:
    Actual: 224
    Invalid: false
    Maximum: 5000
    Percentage: 4.48
    Unit: byte


Once again, the `--best` flag tightens the bounds by traversing each path through the contract and the `--verbose` flag shows the worst-case step in the contract.

## Check whether the execution cost of any contract step exceeds the ledger's maximum budget.

In [21]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --execution-cost

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 3 execution paths.
- Execution cost:
    Memory:
      Actual: 14040278
      Invalid: true
      Maximum: 14000000
      Percentage: 100.2877
    Steps:
      Actual: 3909683086
      Invalid: false
      Maximum: 10000000000
      Percentage: 39.09683086


Check whether the transaction size of any contract step exceeds the ledger's maximum budget.

## Check whether the transaction size exceeds the ledger's limit.

In [22]:
marlowe-cli run analyze \
  --marlowe-file repair-1.marlowe \
  --transaction-size

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 3 execution paths.
- Transaction size:
    Actual: 19256
    Invalid: true
    Maximum: 16384
    Percentage: 117.529296875
